<a href="https://colab.research.google.com/github/careywyr/dlsyscourse/blob/main/homeworks/hw2/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-714 Homework 2

在本次作业中，你将使用 needle 框架实现一个神经网络库。请记得将副本保存到云盘。

In [1]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
/content/drive/MyDrive


In [2]:
# %cd /content/drive/MyDrive/
# !mkdir -p 10714
# %cd /content/drive/MyDrive/10714
# !git clone https://github.com/careywyr/dlsyscourse.git
%cd /content/drive/MyDrive/10714/dlsyscourse/homeworks/hw2

/content/drive/MyDrive/10714
Cloning into 'dlsyscourse'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 153 (delta 33), reused 127 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (153/153), 27.86 MiB | 11.52 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Updating files: 100% (73/73), done.
/content/drive/MyDrive/10714/dlsyscourse/homeworks/hw2


## 问题 0

本次作业是在第一次作业的基础上进行的。首先，在你的作业 2 目录中，将第一次作业中的文件 `python/needle/autograd.py` 和 `python/needle/ops/ops_mathematic.py` 复制过来。

***注意***：张量的默认数据类型为 `float32`。如果需要更改数据类型，可以在 `Tensor` 构造函数中设置 `dtype` 参数。例如，`Tensor([1, 2, 3], dtype='float64')` 将会创建一个数据类型为 `float64` 的张量。在本次作业中，**请确保创建的所有张量均为 `float32` 数据类型，以避免自动评分器产生问题**。

In [3]:
import sys
sys.path.append('./python')
sys.path.append('./apps')

## 问题 1

在第一小题中，你需要实现几种不同的权重初始化方法。这些方法将写入文件 `python/needle/init/init_initializers.py`，该文件包含多个程序，用于通过各种随机和常量初始化方法来初始化 needle 张量。请按照现有的初始化器方法（例如在下面的函数中调用 `python/needle/init/init_basic.py` 中的 `init.rand` 或 `init.randn`），实现以下常见的初始化方法。所有函数均应返回 `fan_in` × `fan_out` 的二维张量（可以通过重塑操作扩展到其他尺寸）。

### Xavier 均匀分布
`xavier_uniform(fan_in, fan_out, gain=1.0, **kwargs)`

根据 [Understanding the difficulty of training deep feedforward neural networks](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) 中的方法，用均匀分布填充输入张量。填充值将从 $\mathcal{U}(-a, a)$ 中采样，其中
\begin{equation}
a = \text{gain} \times \sqrt{\frac{6}{\text{fan_in} + \text{fan_out}}}
\end{equation}

将其余 `**kwargs` 参数传递给对应的 `init` 随机调用。

##### 参数
- `fan_in` - 输入维度
- `fan_out` - 输出维度
- `gain` - 可选的缩放因子
___

### Xavier 正态分布
`xavier_normal(fan_in, fan_out, gain=1.0, **kwargs)`

根据 [Understanding the difficulty of training deep feedforward neural networks](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) 中的方法，用正态分布填充输入张量。填充值将从 $\mathcal{N}(0, \text{std}^2)$ 中采样，其中
\begin{equation}
\text{std} = \text{gain} \times \sqrt{\frac{2}{\text{fan_in} + \text{fan_out}}}
\end{equation}

##### 参数
- `fan_in` - 输入维度
- `fan_out` - 输出维度
- `gain` - 可选的缩放因子
___

### Kaiming 均匀分布
`kaiming_uniform(fan_in, fan_out, nonlinearity="relu", **kwargs)`

根据 [Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification](https://arxiv.org/pdf/1502.01852.pdf) 中的方法，用均匀分布填充输入张量。填充值将从 $\mathcal{U}(-\text{bound}, \text{bound})$ 中采样，其中
\begin{equation}
\text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan_in}}}
\end{equation}

对于 ReLU，建议使用增益值 $\text{gain}=\sqrt{2}$。

##### 参数
- `fan_in` - 输入维度
- `fan_out` - 输出维度
- `nonlinearity` - 非线性函数
___

### Kaiming 正态分布
`kaiming_normal(fan_in, fan_out, nonlinearity="relu", **kwargs)`

根据 [Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification](https://arxiv.org/pdf/1502.01852.pdf) 中的方法，用正态分布填充输入张量。填充值将从 $\mathcal{N}(0, \text{std}^2)$ 中采样，其中
\begin{equation}
\text{std} = \frac{\text{gain}}{\sqrt{\text{fan_in}}}
\end{equation}

对于 ReLU，建议使用增益值 $\text{gain}=\sqrt{2}$。

##### 参数
- `fan_in` - 输入维度
- `fan_out` - 输出维度
- `nonlinearity` - 非线性函数

In [ ]:
!python3 -m pytest -v -k "test_init"

## 问题 2

在本题中，你将需要在 `python/needle/nn/nn_basic.py` 中实现更多的模块。特别是对于以下描述的模块，请在构造函数中初始化该模块的所有变量，并填写 `forward` 方法。**注意**：请务必使用刚实现的 `init` 函数来初始化参数，并确保传递 `dtype` 参数。
___

### Linear
`needle.nn.Linear(in_features, out_features, bias=True, device=None, dtype="float32")`

对输入数据应用线性变换：$y = xA^T + b$。输入的形状为 $(N, H_{in})$，其中 $H_{in}=\text{in_features}$；输出的形状为 $(N, H_{out})$，其中 $H_{out}=\text{out_features}$。

**注意需要显式地将偏置项广播到正确的形状，因为 Needle 不支持隐式广播。**

**注意：对于包括此层的所有层，在初始化时应优先初始化权重张量，再初始化偏置张量，并且所有参数均应只使用 `init` 中的函数进行初始化**。这不会影响算法的正确性，主要是为了确保在本次作业的 mugrade 测试中实现的结果值符合预期。

##### 参数
- `in_features` - 每个输入样本的大小
- `out_features` - 每个输出样本的大小
- `bias` - 如果设置为 `False`，则该层不会学习一个加性偏置。

##### 变量
- `weight` - 可学习的权重，形状为 (`in_features`, `out_features`)。权重的值应使用 `fan_in = in_features` 的 Kaiming 均匀初始化。
- `bias` - 可学习的偏置，形状为 (`out_features`)。偏置的值应使用 `fan_in = out_features` 的 Kaiming 均匀初始化。**注意 `fan_in` 的选择不同，这是由于它们的相对尺寸不同**。

确保将所有必要的变量，例如 (`weight`, `bias`)，放入 `Parameter` 类中，以便它们能被接下来实现的优化器访问。


In [ ]:
!python3 -m pytest -v -k "test_nn_linear"

### ReLU
`needle.nn.ReLU()`

对输入逐元素应用修正线性单元函数：
$ReLU(x) = \max(0, x)$。

如果你之前在实现 ReLU 的反向传播时使用了 ReLU 本身的值，请注意这种方式数值上不稳定，可能会在后续步骤中引发问题。更稳定的做法是将 ReLU 的导数表示为 $I\{x>0\}$，其中我们约定 $x=0$ 时的导数为 0。
（这是一个 _次可微_ 函数。）


In [ ]:
!python3 -m pytest -v -k "test_nn_relu"

### Sequential
`needle.nn.Sequential(*modules)`

按传递给构造函数的顺序依次将多个模块应用于输入，并返回最后一个模块的输出。所有模块应保存在 `.module` 属性中：请勿重新定义诸如 `__getitem__` 等魔术方法，否则可能会与测试不兼容。

##### 参数
- `*modules` - 任意数量的 `needle.nn.Module` 类型模块

___

In [ ]:
!python3 -m pytest -v -k "test_nn_sequential"

### LogSumExp

`needle.ops.LogSumExp(axes)`

对输入应用数值稳定的对数和指数和操作，通过先减去最大元素来实现。你需要在文件 `python/needle/ops/ops_logarithmic.py` 中实现此操作及下一个操作。

\begin{equation}
\text{LogSumExp}(z) = \log (\sum_{i} \exp (z_i - \max{z})) + \max{z}
\end{equation}

#### 参数
- `axes` - 指定需要求和和取最大值的轴，格式为元组。此参数的用法与 `needle.ops.Summation()` 中相同。


___

In [ ]:
!python3 -m pytest -v -k "test_op_logsumexp"

### LogSoftmax

`needle.ops.LogSoftmax(axes)`

通过减去最大元素来对输入应用数值稳定的 LogSoftmax 函数。假设输入 NDArray 为二维，并在 `axis=1` 上执行 softmax 操作。

\begin{equation}
\text{LogSoftmax}(z) = \log \left(\frac{\exp(z_i - \max z)}{\sum_{i}\exp(z_i - \max z)}\right) = z - \text{LogSumExp}(z)
\end{equation}
___

In [ ]:
!python3 -m pytest -v -k "test_op_logsoftmax"

### SoftmaxLoss

`needle.nn.SoftmaxLoss()`

应用以下定义的 softmax 损失（在第一次作业中已实现），输入为 logits 张量和真实标签张量（以数字列表表示，而非 one-hot 编码）。

请注意，现在可以直接使用 `init.one_hot` 函数，而无需自行实现。此外，需要使用刚刚实现的数值稳定的 logsumexp 运算符来完成此操作。


\begin{equation}
\ell_\text{softmax}(z,y) = \log \sum_{i=1}^k \exp z_i - z_y
\end{equation}

___

In [ ]:
!python3 -m pytest -v -k "test_nn_softmax_loss"

### LayerNorm1d
`needle.nn.LayerNorm1d(dim, eps=1e-5, device=None, dtype="float32")`

对输入的小批量数据应用层归一化，如论文 [Layer Normalization](https://arxiv.org/abs/1607.06450) 所描述。

\begin{equation}
y = w \circ \frac{x_i - \textbf{E}[x]}{((\textbf{Var}[x]+\epsilon)^{1/2})} + b
\end{equation}

其中 $\textbf{E}[x]$ 表示输入的经验均值，$\textbf{Var}[x]$ 表示其经验方差（注意，这里使用“有偏”方差估计，即除以 $N$ 而非 $N-1$），$w$ 和 $b$ 分别表示可学习的标量权重和偏置。可以假设此层的输入为二维张量，其中批次在第一个维度，特征在第二个维度。应用权重和偏置前，可能需要对它们进行广播。

##### 参数
- `dim` - 通道数
- `eps` - 为数值稳定性而添加到分母中的值。

##### 变量
- `weight` - 可学习的权重，大小为 `dim`，初始值为 1。
- `bias` - 可学习的偏置，大小为 `dim`，初始值为 0。

___

In [ ]:
!python3 -m pytest -v -k "test_nn_layernorm"


### Flatten
`needle.nn.Flatten()`

接收形状为 `(B, X_0, X_1, ...)` 的张量，并将所有非批次维度展平，使输出形状为 `(B, X_0 * X_1 * ...)`。


In [ ]:
!python3 -m pytest -v -k "test_nn_flatten"

### BatchNorm1d
`needle.nn.BatchNorm1d(dim, eps=1e-5, momentum=0.1, device=None, dtype="float32")`

对小批量数据应用批归一化，如论文 [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167) 所述。

\begin{equation}
y = w \circ \frac{z_i - \textbf{E}[x]}{((\textbf{Var}[x]+\epsilon)^{1/2})} + b
\end{equation}

这里的均值和方差是指在批次维度上的均值和方差。该函数还会计算每层特征的运行均值 $\hat{\mu}$ 和方差 $\hat{\sigma}^2$，并在测试时通过这些值进行归一化：

\begin{equation}
y = \frac{(x - \hat{\mu})}{((\hat{\sigma}^2_{i+1})_j+\epsilon)^{1/2}}
\end{equation}

在测试阶段（即调用 `model.eval()` 将 BatchNorm 层的 `training` 标志设置为 false 后），BatchNorm 使用运行均值和方差，而非批次统计值。

运行均值和方差的计算公式为 $$\hat{x_{new}} = (1 - m) \hat{x_{old}} + mx_{observed},$$ 其中 $m$ 为动量。

##### 参数
- `dim` - 输入维度
- `eps` - 添加到分母中的数值，用于数值稳定性。
- `momentum` - 用于计算运行均值和方差的动量值。

##### 变量
- `weight` - 可学习的权重，大小为 `dim`，初始值为 1。
- `bias` - 可学习的偏置，大小为 `dim`，初始值为 0。
- `running_mean` - 测试时使用的运行均值，初始值为 0。
- `running_var` - 测试时使用的运行（无偏）方差，初始值为 1。


___

In [ ]:
!python3 -m pytest -v -k "test_nn_batchnorm"

### Dropout
`needle.nn.Dropout(p = 0.5)`

在训练期间，Dropout 模块通过从伯努利分布采样，以概率 `p` 随机将输入张量的部分元素置零。这种方法已被证明是一种有效的正则化技术，能够防止神经元的协同适应，详见论文 [Improving neural networks by preventing co-adaption of feature detectors](https://arxiv.org/abs/1207.0580)。在评估期间，模块不执行 Dropout 操作，仅返回输入。

\begin{equation}
\hat{z}_{i+1} = \sigma_i (W_i^T z_i + b_i) \\
(z_{i+1})_j =
    \begin{cases}
    (\hat{z}_{i+1})_j /(1-p) & \text{with probability } 1-p \\
    0 & \text{with probability } p \\
    \end{cases}
\end{equation}

**重要**：如果 Dropout 模块的 `training` 标志设置为 False，则不应“丢弃”任何权重，即 Dropout 仅在训练期间启用，不适用于评估期间。请注意，`training` 是 `nn.Module` 中的一个标志。

##### 参数
- `p` - 元素被置零的概率。


___

In [ ]:
!python3 -m pytest -v -k "test_nn_dropout"

### Residual
`needle.nn.Residual(fn: Module)`

对输入张量 $x$ 应用残差或跳跃连接，给定模块 $\mathcal{F}$，返回 $\mathcal{F}(x) + x$。

##### 参数
- `fn` - `needle.nn.Module` 类型的模块


In [ ]:
!python3 -m pytest -v -k "test_nn_residual"

## 问题 3

在 `python/needle/optim.py` 中实现以下优化器的 `step` 函数。确保优化器**不会**在原地修改张量的梯度。

我们提供了一些测试，以确保不会出现过高的内存消耗。如果没有在合适的位置使用 `.data` 或 `.detach()`，则会生成越来越大的计算图（不仅在优化器中，在前面的模块中也是如此）。对于包含字符串 `memory_check` 的测试，你可以自行决定是否忽略。

___

### SGD
`needle.optim.SGD(params, lr=0.01, momentum=0.0, weight_decay=0.0)`

实现随机梯度下降（可选动量 $\beta$）：

\begin{equation}
\begin{split}
    u_{t+1} &= \beta u_t + (1-\beta) \nabla_\theta f(\theta_t) \\
    \theta_{t+1} &= \theta_t - \alpha u_{t+1}
\end{split}
\end{equation}

##### 参数
- `params` - `needle.nn.Parameter` 类型的参数迭代器，用于优化
- `lr` (*float*) - 学习率
- `momentum` (*float*) - 动量因子
- `weight_decay` (*float*) - 权重衰减（L2 正则化项）

___

In [ ]:
!python3 -m pytest -v -k "test_optim_sgd"

### Adam
`needle.optim.Adam(params, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8, weight_decay=0.0)`

实现 Adam 算法，详见论文 [Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980)。

\begin{equation}
\begin{split}
u_{t+1} &= \beta_1 u_t + (1-\beta_1) \nabla_\theta f(\theta_t) \\
v_{t+1} &= \beta_2 v_t + (1-\beta_2) (\nabla_\theta f(\theta_t))^2 \\
\hat{u}_{t+1} &= u_{t+1} / (1 - \beta_1^t) \quad \text{(偏差校正)} \\
\hat{v}_{t+1} &= v_{t+1} / (1 - \beta_2^t) \quad \text{(偏差校正)}\\
\theta_{t+1} &= \theta_t - \alpha \hat{u_{t+1}}/(\hat{v}_{t+1}^{1/2}+\epsilon)
\end{split}
\end{equation}

**重要**：注意是否正确应用了偏差校正。

##### 参数
- `params` - `needle.nn.Parameter` 类型的参数迭代器，用于优化
- `lr` (*float*) - 学习率
- `beta1` (*float*) - 用于计算梯度运行平均的系数
- `beta2` (*float*) - 用于计算梯度平方运行平均的系数
- `eps` (*float*) - 为提升数值稳定性而添加到分母的值
- `weight_decay` (*float*) - 权重衰减（L2 正则项）

**提示**：为减少内存占用，尝试使用 `.data` 或 `.detach()`。


In [ ]:
!python3 -m pytest -v -k "test_optim_adam"

## 问题 4

本题中，你将实现两个数据基础类：`needle.data.DataLoader` 和 `needle.data.Dataset`。`Dataset` 用于存储样本及其对应标签，而 `DataLoader` 则将 `Dataset` 包装为一个可迭代对象，便于访问样本。

本题要求你在 `python/needle/data` 目录下进行实现。

### 转换

首先，我们将实现一些图像处理中的常用转换，目前包括水平翻转和随机裁剪。在 `needle/data/data_transforms.py` 中填写以下函数。
___

#### RandomFlipHorizontal
`needle.data.RandomFlipHorizontal(p = 0.5)`

以概率 `p` 将图像进行水平翻转。

##### 参数
- `p` (*float*) - 图像水平翻转的概率。
___

#### RandomCrop
`needle.data.RandomCrop(padding=3)`

在图像的每一边添加填充后，在随机位置将图像裁剪回原始大小。输出图像大小与原图相同。

##### 参数
- `padding` (*int*) - 图像每个边缘的填充大小。


In [ ]:
!python3 -m pytest -v -k "flip_horizontal"
!python3 -m pytest -v -k "random_crop"

### Dataset

每个 `Dataset` 子类必须实现 `__init__`、`__len__` 和 `__getitem__` 三个函数。`__init__` 函数初始化图像、标签和转换操作。`__len__` 返回数据集中样本数量，`__getitem__` 则在给定索引 `idx` 处获取数据集样本，对图像应用转换函数（如果适用），并将图像和标签转换为 numpy 数组（数据将在其他部分转换为 Tensor）。`__getitem__` 和 `__next__` 的输出应为大小为 (样本数, 特征维度1, 特征维度2, ...) 的 NDArray。

在 `needle/data/datasets/mnist_dataset.py` 中的 `MNISTDataset` 类中实现这些函数。`__init__` 函数可以使用上次作业中的 `parse_mnist` 方法。

### MNISTDataset
`needle.data.MNISTDataset(image_filesname, label_filesname, transforms)`

##### 参数
- `image_filesname` - 图像文件的路径
- `label_filesname` - 标签文件的路径
- `transforms` - 可选的转换列表，用于数据转换

In [ ]:
!python3 -m pytest -v -k "test_mnist_dataset"

### Dataloader

`needle/data/data_basic.py` 中的 Dataloader 类提供了一个接口，用于将样本组装为适合 SGD 训练的小批量数据，基于 Dataset 对象。要构建标准的 Dataloader 接口（允许用户遍历数据集中的所有小批量），需要在类中实现 `__iter__()` 和 `__next__()` 方法：`__iter__()` 在迭代开始时调用，而 `__next__()` 则获取下一个小批量。注意，`next` 后续调用需返回后续小批量，因此不是纯函数。

### Dataloader
`needle.data.Dataloader(dataset: Dataset, batch_size: Optional[int] = 1, shuffle: bool = False)`

将数据集与采样器组合，并提供对数据集的可迭代访问。

##### 参数
- `dataset` - `needle.data.Dataset` - 数据集对象
- `batch_size` - `int` - 批次大小
- `shuffle` - `bool` - 每个 epoch 是否打乱数据，默认值为 ``False``。
___



In [ ]:
!python3 -m pytest -v -k "test_dataloader"

## 问题 5

现在你已经实现了所有必要的神经网络库组件，我们可以构建并训练一个 MLP ResNet。在本题中，请在 `apps/mlp_resnet.py` 文件中实现 `ResidualBlock` 和 `MLPResNet` 函数。

### ResidualBlock
`ResidualBlock(dim, hidden_dim, norm=nn.BatchNorm1d, drop_prob=0.1)`

实现如下所示的残差块：

<p align="center">
  <img src="https://raw.github.com/dlsyscourse/hw2/blob/main/figures/residualblock.png" alt="Residual Block"/>
</p>

**注意**：如果图像未显示，请查看 `figures` 目录中的图像。

该模块的第一个线性层的 `in_features=dim`，`out_features=hidden_dim`，最后一个线性层的 `out_features=dim`。返回类型为 `nn.Module`。

##### 参数
- `dim` (*int*) - 输入维度
- `hidden_dim` (*int*) - 隐藏层维度
- `norm` (*nn.Module*) - 归一化方法
- `drop_prob` (*float*) - dropout 概率

___

### MLPResNet
`MLPResNet(dim, hidden_dim=100, num_blocks=3, num_classes=10, norm=nn.BatchNorm1d, drop_prob=0.1)`

实现如下所示的 MLP ResNet：

<p align="center">
  <img src="https://raw.github.com/dlsyscourse/hw2/blob/main/figures/mlp_resnet.png" alt="MLP Resnet"/>
</p>

第一个线性层的 `in_features=dim`，`out_features=hidden_dim`。每个 ResidualBlock 的 `dim=hidden_dim`，`hidden_dim=hidden_dim//2`。返回网络类型为 `nn.Module`。

##### 参数
- `dim` (*int*) - 输入维度
- `hidden_dim` (*int*) - 隐藏层维度
- `num_blocks` (*int*) - 残差块的数量
- `num_classes` (*int*) - 类别数
- `norm` (*nn.Module*) - 归一化方法
- `drop_prob` (*float*) - dropout 概率 (0.1)

**注意**：模块应按 ResNet 中的执行顺序进行初始化。
___

完成模型架构后，使用新的神经网络库组件来训练网络。具体来说，实现 `epoch` 和 `train_mnist` 函数。

### Epoch

`epoch(dataloader, model, opt=None)`

执行一个训练或评估 epoch，遍历整个训练数据集一次（如之前作业中的 `nn_epoch`）。返回平均错误率（*float*）和所有样本的平均损失（*float*）。若 `opt` 存在，则在函数开始时将模型设为 `training` 模式；若 `opt` 为 `None`，则设为 `eval` 模式。请使用 `.train()` 和 `.eval()` 设置模式，而不是直接修改 training 属性。

##### 参数
- `dataloader` (*`needle.data.DataLoader`*) - 返回训练样本的数据加载器
- `model` (*`needle.nn.Module`*) - 神经网络
- `opt` (*`needle.optim.Optimizer`*) - 优化器实例，或 `None`

___

### 训练 MNIST

`train_mnist(batch_size=100, epochs=10, optimizer=ndl.optim.Adam, lr=0.001, weight_decay=0.001, hidden_dim=100, data_dir="data")`

初始化用于 MNIST 数据的训练数据加载器（`shuffle=True`）和测试数据加载器，并使用指定优化器（若 `opt` 存在）和 softmax 损失函数进行训练。返回一个元组，包括最后一个 epoch 的训练错误率、训练损失、测试错误率和测试损失。若参数未指定，请使用默认值。

##### 参数
- `batch_size` (*int*) - 训练和测试数据加载器的批次大小
- `epochs` (*int*) - 训练 epoch 数
- `optimizer` (*`needle.optim.Optimizer` 类型*) - 使用的优化器类型
- `lr` (*float*) - 学习率
- `weight_decay` (*float*) - 权重衰减
- `hidden_dim` (*int*) - MLPResNet 的隐藏层维度
- `data_dir` (*str*) - 包含 MNIST 图像/标签的目录


In [ ]:
!python3 -m pytest -v -k "test_mlp"

我们鼓励你尝试使用 `mlp_resnet.py` 训练脚本。你可以观察不同初始化方法在 Linear 层中的效果、提高 dropout 概率，或通过 Dataset 的 `transforms=` 参数添加转换（如随机裁剪）。